In [1]:
!pip install --upgrade --quiet pip
!pip install --quiet farm-haystack[colab,faiss]==1.17.2
print('pip install haystack complete.')

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
chex 0.1.82 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
google-cloud-aiplatform 0.6.0a1 requires google-api-core[grpc]<2.0.0dev,>=1.22.2, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-automl 1.0.1 requires google-api-core[grpc]<2.0.0dev,>=1.14.0, but you have google-api-core 2.11.1 which is incompatible.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.52 which is incompatible.
kfp 2.0.1 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, but you have numpy 1.23.5 which is incompatible.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you have scip

In [2]:
!apt-get install --quiet poppler-utils -y
print('pip install poppler utils complete.')
!sudo apt-get update
!sudo apt-get install --quiet poppler-utils -y
print('pip install poppler complete for real and for true')

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libpoppler97 poppler-data
Suggested packages:
  ghostscript fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  libpoppler97 poppler-data poppler-utils
0 upgraded, 3 newly installed, 0 to remove and 40 not upgraded.
Need to get 2562 kB of archives.
After this operation, 16.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 poppler-data all 0.4.9-2 [1475 kB]
Ign:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libpoppler97 amd64 0.86.1-0ubuntu1.3
Ign:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 poppler-utils amd64 0.86.1-0ubuntu1.3
Err:2 http://security.ubuntu.com/ubuntu focal-updates/main amd64 libpoppler97 amd64 0.86.1-0ubuntu1.3
  404  Not

In [3]:
import os
import requests

URL = ['https://www.senate.gov/civics/resources/pdf/US_Constitution-Senate_Publication_103-21.pdf',
       'https://www.govinfo.gov/content/pkg/GPO-CONAN-2017/pdf/GPO-CONAN-2017.pdf'][-1]
WORKING = '/kaggle/working/'

# https://stackoverflow.com/a/56951135
def download(url: str, dest_folder: str):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)  # create folder if it does not exist

    filename = url.split('/')[-1].replace(" ", "_")  # be careful with file names
    file_path = os.path.join(dest_folder, filename)

    r = requests.get(url, stream=True)
    if r.ok:
        print("saving to", os.path.abspath(file_path))
        with open(file=file_path, mode='wb') as f:
            for chunk in r.iter_content(chunk_size=1024 * 8):
                if chunk:
                    f.write(chunk)
                    f.flush()
                    os.fsync(f.fileno())
    else:  # HTTP status code 4XX/5XX
        print("Download failed: status code {}\n{}".format(r.status_code, r.text))


short_name = os.path.basename(URL)
if not os.path.exists(WORKING + short_name):
    download(url=URL, dest_folder=WORKING)
    print('downloaded {} to {}'.format(URL, WORKING))
else:
    print('file {} already exists.'.format(WORKING + short_name))
pdf_paths=[WORKING + short_name]

saving to /kaggle/working/GPO-CONAN-2017.pdf
downloaded https://www.govinfo.gov/content/pkg/GPO-CONAN-2017/pdf/GPO-CONAN-2017.pdf to /kaggle/working/


In [4]:
from haystack.document_stores import FAISSDocumentStore

document_store = FAISSDocumentStore(embedding_dim=128, faiss_index_factory_str='Flat')
print('created our document store')

created our document store


In [5]:
# Initialize a PDFToTextConverter to convert the PDF to text
from haystack.utils import convert_files_to_docs, fetch_archive_from_http, clean_wiki_text
from haystack.utils import fetch_archive_from_http, convert_files_to_docs
from haystack.nodes import PDFToTextConverter, PreProcessor, DensePassageRetriever
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor
print('many imports complete.')

many imports complete.


In [6]:
#replace this paths with your input Data path

# Convert the PDF documents to Haystack-compatible format
pdf_converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=['en'])
pdf_docs = []

for pdf_path in pdf_paths:
    doc = pdf_converter.convert(file_path=pdf_path, meta=None)[0]
    pdf_docs.append(doc)
print('converted {} documents'.format(len(pdf_docs)))

pdftotext version 0.86.1
Copyright 2005-2020 The Poppler Developers - http://poppler.freedesktop.org
Copyright 1996-2011 Glyph & Cog, LLC


converted 1 documents


In [7]:
# Initializing the PreProcessor to clean and split the text
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by='word',
    split_length=100,
    split_respect_sentence_boundary=True,
)
processed_docs = preprocessor.process(pdf_docs)
print('document processing done.')

Preprocessing:   0%|          | 0/1 [00:00<?, ?docs/s]

document processing done.


In [8]:
# Initialize a DensePassageRetriever
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model='vblagoje/dpr-question_encoder-single-lfqa-wiki',
    passage_embedding_model='vblagoje/dpr-ctx_encoder-single-lfqa-wiki',
)
print('built retriever.')

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


built retriever.


In [9]:
document_store.write_documents(processed_docs)
document_store.update_embeddings(retriever)
print('updated embeddings.')

Writing Documents:   0%|          | 0/14909 [00:00<?, ?it/s]

Updating Embedding:   0%|          | 0/14895 [00:00<?, ? docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/4896 [00:00<?, ? Docs/s]

updated embeddings.


In [10]:
from haystack.nodes import Seq2SeqGenerator

generator = Seq2SeqGenerator(model_name_or_path='vblagoje/bart_lfqa')
print('built generator.')

built generator.


In [11]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path='deepset/deberta-v3-large-squad2')
print('built the reader.')

built the reader.


In [12]:
from haystack.pipelines import GenerativeQAPipeline
from haystack.pipelines import ExtractiveQAPipeline

generative_pipeline = GenerativeQAPipeline(generator, retriever)
extractive_pipeline = ExtractiveQAPipeline(reader, retriever)

In [13]:
query = 'What is a summary of this document?'
result = extractive_pipeline.run(query=query, params={'Retriever': {'top_k': 1}, 'Reader': {'top_k': 1}})
print(result)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

{'query': 'What is a summary of this document?', 'no_ans_gap': 12.300952196121216, 'answers': [<Answer {'answer': '1051\nAmendments to the Constitution', 'type': 'extractive', 'score': 0.914199709892273, 'context': '......................................................... 1051\nAmendments to the Constitution ........................................................', 'offsets_in_document': [{'start': 1788, 'end': 1823}], 'offsets_in_context': [{'start': 58, 'end': 93}], 'document_ids': ['a2cd4939e575f1852b4142c0c981e23c'], 'meta': {'_split_id': 105, 'vector_id': '9075'}}>], 'documents': [<Document: {'content': 'v\nIntroduction to the 2012 Centennial Edition .......................................................................... ix\nHistorical note on formation of the Constitution .................................................................... xvii\nText of the Constitution (literal print) ..................................................................................... 1\nT

In [14]:
print(result['answers'][0].answer)

1051
Amendments to the Constitution


In [15]:
piperun=generative_pipeline.run(
    query='What is this book about?', params={'Retriever': {'top_k': 5}} 
)
for answer in piperun['answers']:
    print(answer.answer)

I'm not sure what you're looking for, but you might be interested in this previous answer by /u/commiespaceinvader.


In [16]:
piperun=generative_pipeline.run(
    query='Who was Edward Said?', params={'Retriever': {'top_k': 5}} 
)
for answer in piperun['answers']:
    print(answer.answer)

Edward Said was a British philosopher who wrote in the late 19th and early 20th century. He was one of the most influential thinkers of his time. He wrote a number of books on the history of science, philosophy, and literature. His most famous work was *The Structure of Scientific Revolutions*, which is a collection of essays written in the 19th century on the subject of science and philosophy. He also wrote *The Cambridge Companion to the History of Science*, which was published by Cambridge University Press in the mid-20th century, and published by Penguin Books in the early 21st century.


In [17]:
piperun=generative_pipeline.run(
    query='Where did Edward Said go to school?', params={'Retriever': {'top_k': 5}} 
)
for answer in piperun['answers']:
    print(answer.answer)

Edward Said was born and raised in London, England. He was educated in an Anglican school, and went on to study at the University of Cambridge, where he was a member of the Cambridge School of Oriental and African Studies.


In [18]:
piperun=generative_pipeline.run(
    query='What countries are mentioned in this book?', params={'Retriever': {'top_k': 5}} 
)
for answer in piperun['answers']:
    print(answer.answer)

I'm not a historian, but I do have a degree in history, so I'll try to answer your question. I'm not sure if this is what you're looking for, but you might be interested in this previous answer by u/commiespaceinvader.


In [19]:
piperun=generative_pipeline.run(
    query='What are the major themes of this book?', params={'Retriever': {'top_k': 5}} 
)
for answer in piperun['answers']:
    print(answer.answer)

I'm not sure what you mean by "major themes," but I'll give it a shot. I'm not a historian, but I do have a degree in history, so I'll try to answer your question as best as I can. The main themes of the book are: 1. How did the Constitution come to be? 2. What was the purpose of the Constitution? 3. Why did it become the law of the land? 4. What did it mean to be a citizen of the United States? 5. What happened to the Constitution when it was written? 6. What is the meaning of the Declaration of Independence? 7. What does the Constitution mean to the Founding Fathers?


In [20]:
piperun=generative_pipeline.run(
    query='Please tell me everything about Edward Said.', params={'Retriever': {'top_k': 5}} 
)
for answer in piperun['answers']:
    print(answer.answer)

Edward Said is one of the most important thinkers of the 20th century. He wrote a number of seminal works on the history of the modern world, including *The Origins of Modernity* and *The Rise and Fall of the Modern World*. He also wrote a book called *A History of the World*, which is a collection of essays written by people from all over the world.


In [21]:
piperun=generative_pipeline.run(
    query='What did Edward Said think of the Balfour Declaration?', params={'Retriever': {'top_k': 2}} 
)
for answer in piperun['answers']:
    print(answer.answer)

Edward Said was not a fan of the Balfour Declaration. He was a staunch supporter of the British Empire, and saw the Declaration as an attempt to undermine British power in the world. He believed that the Declaration was an attempt by the British government to weaken the United States, and that it was not intended to be a declaration of independence.


In [22]:
piperun=generative_pipeline.run(
    query='Where was Edward Said in 1991?', params={'Retriever': {'top_k': 5}} 
)
for answer in piperun['answers']:
    print(answer.answer)

I'm not sure if this is what you're looking for, but here's an article from the New York Times about Edward Said in 1991.


In [23]:
piperun=generative_pipeline.run(
    query='Did the committee conclude that the CIA detention and interrogation program was ethical?', params={'Retriever': {'top_k': 5}} #output varies with different top_k values
)
for answer in piperun['answers']:
    print(answer.answer)

No, the committee did not conclude that the CIA detention and interrogation program was ethical.


In [24]:
piperun=generative_pipeline.run(
    query='What is Edward Said\'s legacy?', params={'Retriever': {'top_k': 5}}
)
piperun['answers'][0].answer

"Edward Said's legacy is that he was one of the first, if not THE first, to write a book about the Holocaust. His book, *The Holocaust: An Oral History*, was published in 1948, and it was the first book to be published in English. It was also the first to be translated into English, and the first English-language book to have been published in the United States. The book was published by Penguin Books, which was a British publishing house, and was translated into several languages, including English, German, French, and Italian. It is widely considered to be the definitive work on the Holocaust, and has been translated into more than a dozen languages."